<a href="https://colab.research.google.com/github/aaronayitey/Sentiment-Analysis-Covid_19-Twitter-Reactions/blob/main/Model_ROBERTA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install huggingface_hub transformers datasets gradio pipreqs TextBlob emot xformers transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 111.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (se

In [4]:
!pip install typing-extensions<4.6.0

/bin/bash: line 1: 4.6.0: No such file or directory


In [5]:
!pip install datasets huggingface_hub -q

In [55]:
# Import key libraries and packages
import streamlit
import numpy as np
import os
import uuid
import pandas as pd

from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainingArguments, Trainer
from transformers import IntervalStrategy
from transformers import EarlyStoppingCallback
from transformers import pipeline
from transformers import TrainingArguments
from transformers import Trainer
from torch import nn
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification

In [32]:
notebook_login()

In [33]:

# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

In [34]:
# Load the CSV file into a DataFrame

url = "https://github.com/Azubi-Africa/Career_Accelerator_P5-NLP/raw/master/zindi_challenge/data/Train.csv"

df_train = pd.read_csv(url)

In [35]:
df_train.head()

,tweet_id,safe_text,label,agreement
0,CL1KWCMY,Me &amp; The Big Homie meanboy3000 #MEANBOY #M...,0.0,1.0
1,E3303EME,I'm 100% thinking of devoting my career to pro...,1.0,1.0
2,M4IVFSMS,"#whatcausesautism VACCINES, DO NOT VACCINATE Y...",-1.0,1.0
3,1DR6ROZ4,I mean if they immunize my kid with something ...,-1.0,1.0
4,J77ENIIE,Thanks to <user> Catch me performing at La Nui...,0.0,1.0


In [36]:
# Select rows with missing values
df_train.isnull().sum()

tweet_id     0
safe_text    0
label        1
agreement    2
dtype: int64

In [37]:
# Select rows with missing values
df_train[df_train.isnull().any(axis=1)]

,tweet_id,safe_text,label,agreement
4798,RQMQ0L2A,#lawandorderSVU,NaN,NaN
4799,I cannot believe in this day and age some pare...,1,0.666667,NaN


In [38]:
# Extract complete text from 'safe_text' column
complete_text = df_train.iloc[4798]['safe_text']
complete_text

'#lawandorderSVU '

In [39]:
# Select row by index and assign values to columns
df_train.loc[4798, 'label'] = 0
df_train.loc[4798, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df_train.iloc[4798, df_train.columns.get_loc('safe_text')] = complete_text

In [40]:
# Generate random UUID string for tweet_id
'''UUIDs are often used in software applications for various purposes such as generating unique IDs for entities,
tracking unique user sessions, or creating unique file names'''
rand_tweet_id = str(uuid.uuid4())

# Select row by index and assign values to columns
row_index = 4799
df_train.loc[row_index, 'tweet_id'] = rand_tweet_id
df_train.loc[row_index, 'label'] = 1
df_train.loc[row_index, 'agreement'] = 0.666667

# Use .iloc[] and .iat[] to select and update safe_text column
df_train.iloc[row_index, df_train.columns.get_loc('safe_text')] = df_train.iloc[row_index, 1]

In [41]:
df_train['label'].value_counts()

 0.0    4909
 1.0    4054
-1.0    1038
Name: label, dtype: int64

In [43]:
# Find the maximum count among all classes
max_class_count = df_train['label'].value_counts().max()

# Group the dataframe by 'label'
grouped = df_train.groupby('label')

# Sample each group to match the max_class_count
balanced_train = grouped.apply(lambda x: x.sample(max_class_count, replace=True)).reset_index(drop=True)

# The 'balanced_df' now contains an equal number of instances for each class
balanced_train['label'].value_counts()

-1.0    4909
 0.0    4909
 1.0    4909
Name: label, dtype: int64

In [44]:
# Split the train data => {train, eval}
train, eval = train_test_split(balanced_train, test_size=0.2, random_state=42)

In [45]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11781 entries, 2435 to 7270
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   11781 non-null  object 
 1   safe_text  11781 non-null  object 
 2   label      11781 non-null  float64
 3   agreement  11781 non-null  float64
dtypes: float64(2), object(2)
memory usage: 460.2+ KB


In [46]:
eval.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2946 entries, 13730 to 9863
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tweet_id   2946 non-null   object 
 1   safe_text  2946 non-null   object 
 2   label      2946 non-null   float64
 3   agreement  2946 non-null   float64
dtypes: float64(2), object(2)
memory usage: 115.1+ KB


In [47]:
file_path = '/content/drive/MyDrive/NLP/data_sent'
train.to_csv(os.path.join(file_path, "train_subset.csv"), index=False)
eval.to_csv(os.path.join(file_path, "eval_subset.csv"), index=False)

In [48]:
# Load the CSV files into a dataset

dataset = load_dataset('csv', data_files={
    'train': '/content/drive/MyDrive/NLP/data_sent/train_subset.csv',
    'eval': '/content/drive/MyDrive/NLP/data_sent/eval_subset.csv'
}, encoding='ISO-8859-1')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

In [49]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                          # Directory where the model checkpoints and evaluation results will be stored
    evaluation_strategy=IntervalStrategy.STEPS,      # Interval for evaluating the model during training (every specified number of steps)
    save_strategy=IntervalStrategy.STEPS,            # Interval for saving the model during training (every specified number of steps)
    save_steps=500,                                  # Number of steps between two saves
    load_best_model_at_end=True,                     # Whether to load the best model at the end of training
    num_train_epochs=5,                              # Number of training epochs
    per_device_train_batch_size=4,                   # Batch size per GPU for training
    per_device_eval_batch_size=4,                    # Batch size per GPU for evaluation
    learning_rate=3e-5,                              # Learning rate
    weight_decay=0.01,                               # Weight decay
    warmup_steps=500,                                # Number of warmup steps
    logging_steps=500,                               # Number of steps between two logs
    fp16=True,                                       # Whether to use 16-bit precision
    gradient_accumulation_steps=5,                  # Number of steps to accumulate gradients before performing an optimizer step
    dataloader_num_workers=2,                        # Number of workers to use for loading data
    push_to_hub=True,                                # Whether to push the model checkpoints to the Hugging Face hub
    hub_model_id="aaronayitey/Sentiment-classfication-ROBERTA-model",  # Model ID to use when pushing the model to the Hugging Face hub
)



# Define the early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,                       # Number of epochs with no improvement before stopping training
    early_stopping_threshold=0.01,                   # Minimum improvement in the metric for considering an improvement
)

# Combine the training arguments and the early stopping callback
training_args.callbacks = [early_stopping]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [50]:
tokenizer_ROBERTA = AutoTokenizer.from_pretrained('roberta-base')

In [51]:
# Define a function to transform the label values
def transform_labels(label):
    # Extract the label value
    label = label['label']
    # Map the label value to an integer value
    num = 0
    if label == -1: #'Negative'
        num = 0
    elif label == 0: #'Neutral'
        num = 1
    elif label == 1: #'Positive'
        num = 2
    # Return a dictionary with a single key-value pair
    return {'labels': num}

# Define a function to tokenize the text data
def tokenize_data(example):
    # Extract the 'safe_text' value from the input example and tokenize it
    return tokenizer_ROBERTA(example['safe_text'], padding='max_length')

# Apply the transformation functions to the dataset using the 'map' method
# This transforms the label values and tokenizes the text data
dataset_out = dataset.map(transform_labels)

dataset_ROBERTA = dataset_out.map(tokenize_data, batched=True)

# Define a list of column names to remove from the dataset
remove_columns = ['tweet_id', 'label', 'safe_text', 'agreement']

# Apply the 'transform_labels' function to the dataset to transform the label values
# Also remove the columns specified in 'remove_columns'

dataset_ROBERTA = dataset_ROBERTA.map(transform_labels, remove_columns=remove_columns)

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

Map:   0%|          | 0/11781 [00:00<?, ? examples/s]

Map:   0%|          | 0/2946 [00:00<?, ? examples/s]

In [52]:
# Loading a pretrain model while specifying the number of labels in our dataset for fine-tuning
model_ROBERTA = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
train_dataset_ROBERTA = dataset_ROBERTA['train'].shuffle(seed=10)
eval_dataset_ROBERTA = dataset_ROBERTA['eval'].shuffle(seed=10)

In [54]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    rmse = np.sqrt(np.mean((predictions - labels)**2))
    return {"rmse": rmse}

In [56]:
trainer_ROBERTA = Trainer(
    model=model_ROBERTA,
    args=training_args,
    train_dataset=train_dataset_ROBERTA,
    eval_dataset=eval_dataset_ROBERTA,
    compute_metrics=compute_metrics    # Add this line to define the compute_metrics function
)

In [57]:
trainer_ROBERTA.train()

Step,Training Loss,Validation Loss,Rmse
500,0.831300,0.594061,0.699141
1000,0.454000,0.334472,0.459862
1500,0.255800,0.283408,0.351990
2000,0.155300,0.373728,0.359622
2500,0.099000,0.370177,0.320704


TrainOutput(global_step=2945, training_loss=0.314929171369517, metrics={'train_runtime': 2644.3002, 'train_samples_per_second': 22.276, 'train_steps_per_second': 1.114, 'total_flos': 1.5494222943166464e+16, 'train_loss': 0.314929171369517, 'epoch': 5.0})

In [58]:
# Evaluate the model
eval_results = trainer_ROBERTA.evaluate()

# Create a dictionary of the evaluation results
results_dict = {
    "Model": "roberta-base",
    "Loss": eval_results["eval_loss"],
    "RMSE": eval_results["eval_rmse"],
    "Runtime": eval_results["eval_runtime"],
    "Samples Per Second": eval_results["eval_samples_per_second"],
    "Steps Per Second": eval_results["eval_steps_per_second"],
    "Epoch": eval_results["epoch"]
}

# Create a pandas DataFrame from the dictionary
results_df = pd.DataFrame([results_dict])

# Sort the results by "eval_rmse" in ascending order and get the name and state dict of the best model
best_model = results_df.loc[results_df['RMSE'].idxmin()]

best_model

Model                 roberta-base
Loss                      0.283408
RMSE                       0.35199
Runtime                    38.8419
Samples Per Second          75.846
Steps Per Second            18.974
Epoch                          5.0
Name: 0, dtype: object

In [60]:
 # Push the final fine-tuned model to the Hugging Face model hub

trainer_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")
tokenizer_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")
model_ROBERTA.push_to_hub("aaronayitey/Sentiment-classfication-ROBERTA-model")

events.out.tfevents.1699460673.ce228ffa7908.250.0:   0%|          | 0.00/7.12k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1699463437.ce228ffa7908.250.1:   0%|          | 0.00/407 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/aaronayitey/Sentiment-classfication-ROBERTA-model/commit/cba83af57cfcd35681bf0f36276bb642a5422780', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='cba83af57cfcd35681bf0f36276bb642a5422780', pr_url=None, pr_revision=None, pr_num=None)

In [61]:
# Load the tokenizer
tokenizer = tokenizer_ROBERTA.from_pretrained("aaronayitey/Sentiment-classfication-ROBERTA-model")

# Load the fine-tuned model
model = pipeline("text-classification", model="aaronayitey/Sentiment-classfication-ROBERTA-model", tokenizer=tokenizer)


In [70]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

# Make predictions on some example text
result = model("I'm sure not to take in that vaccine.")

# Map the numerical label to the corresponding class name
result[0]["label"] = label_map[int(result[0]["label"].split("_")[1])]

# Print the predicted label and score
result

[{'label': 'negative', 'score': 0.9799835085868835}]